In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd

# Define the Find-S Algorithm
def find_s_algorithm(df, target_product):
    positive_examples = df[df['PurchasedProduct'] == target_product]

    if positive_examples.empty:
        return None

    # Start with the most specific hypothesis (i.e., the first positive example)
    initial_hypothesis = positive_examples.iloc[0].drop(['CustomerID', 'PurchasedProduct'])
    hypothesis = initial_hypothesis.copy()

    for _, example in positive_examples.iterrows():
        for feature in hypothesis.index:
            if hypothesis[feature] != example[feature]:
                hypothesis[feature] = '?'

    return hypothesis

# Define the Candidate Elimination Algorithm
def candidate_elimination(df, target_product):
    num_features = df.shape[1] - 2
    specific_hypothesis = [None] * num_features
    general_hypothesis = [['?' for _ in range(num_features)]]

    def update_specific(example):
        nonlocal specific_hypothesis
        current_example = example.drop(['CustomerID', 'PurchasedProduct'])
        if specific_hypothesis == [None] * num_features:
            specific_hypothesis = current_example.tolist()
        else:
            for i in range(num_features):
                if specific_hypothesis[i] is None or specific_hypothesis[i] == current_example.iloc[i]:
                    specific_hypothesis[i] = current_example.iloc[i]
                else:
                    specific_hypothesis[i] = '?'

    def update_general(example):
        nonlocal general_hypothesis
        current_example = example.drop(['CustomerID', 'PurchasedProduct'])
        new_general_hypothesis = []
        for hypo in general_hypothesis:
            if all(h == '?' or h == v for h, v in zip(hypo, current_example)):
                new_general_hypothesis.append(hypo)
            else:
                new_hypo = hypo[:]
                for i, (h, v) in enumerate(zip(hypo, current_example)):
                    if h != v and h != '?':
                        new_hypo[i] = '?'
                new_general_hypothesis.append(new_hypo)
        general_hypothesis = new_general_hypothesis

    for _, example in df.iterrows():
        if example['PurchasedProduct'] == target_product:
            update_specific(example)
        else:
            update_general(example)

    return specific_hypothesis, general_hypothesis

def main():
    # Set up custom CSS for dark theme and monospaced font
    st.markdown(
        """
        <style>
        /* Dark theme */
        .streamlit-expanderHeader {
            background-color: #2c2c2c;
            color: white;
        }
        .streamlit-expanderContent {
            background-color: #1e1e1e;
        }
        .css-1l02z6r {
            color: white;
        }
        .css-1v0mbdj {
            background-color: #1e1e1e;
        }
        /* Monospaced font */
        body {
            font-family: 'Fira Code', monospace;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    st.title("Product Purchase Prediction")

    st.sidebar.header("Upload Data and Choose Algorithm")

    # File uploader widget
    uploaded_file = st.sidebar.file_uploader("Upload a CSV file", type="csv")

    if uploaded_file is not None:
        # Read CSV file
        df = pd.read_csv(uploaded_file)

        # Ensure required columns are present
        required_columns = {'CustomerID', 'Color', 'Size', 'Brand', 'PurchasedProduct'}
        if not required_columns.issubset(df.columns):
            st.error("CSV file must contain the following columns: CustomerID, Color, Size, Brand, PurchasedProduct")
            return

        st.sidebar.subheader("Algorithm Selection")
        algorithm = st.sidebar.radio("Choose an algorithm:", ("Find-S", "Candidate Elimination"))

        target_product = st.sidebar.text_input("Enter the target product:", "Shoes")

        if target_product:
            if algorithm == "Find-S":
                result = find_s_algorithm(df, target_product)
                st.subheader(f"Find-S Algorithm Results for '{target_product}'")
                if result is not None:
                    st.write(pd.DataFrame([result], columns=df.columns[1:-1]).T, header=False)
                else:
                    st.write("No positive examples found for the target product.")

            elif algorithm == "Candidate Elimination":
                specific_hypothesis, general_hypothesis = candidate_elimination(df, target_product)
                st.subheader(f"Candidate Elimination Results for '{target_product}'")

                st.write("### Specific Hypothesis")
                if specific_hypothesis:
                    st.write(pd.DataFrame([specific_hypothesis], columns=df.columns[1:-1]).T, header=False)
                else:
                    st.write("No specific hypothesis found.")

                st.write("### General Hypotheses")
                if general_hypothesis:
                    st.write(pd.DataFrame(general_hypothesis, columns=df.columns[1:-1]), header=False)
                else:
                    st.write("No general hypotheses found.")

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501


34.48.80.47
Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://late-dodos-swim.loca.lt
